# Deep Agents Experiments: Progressive Learning Journey

**A comprehensive hands-on guide to understanding Deep Agents architecture through progressive experiments**

This notebook transforms the concepts from `deep_agents_notes.md` into executable experiments, allowing you to experience the evolution from basic tool-calling agents to advanced autonomous systems.

## Learning Objectives

By the end of this notebook, you will understand:
- **Agent Evolution**: From reactive tool-callers to proactive autonomous systems
- **Middleware Architecture**: How planning, filesystem, and subagents work together
- **Filesystem Patterns**: Virtual vs real filesystem usage and best practices
- **Performance Optimization**: Cost-effective prompt caching strategies
- **Implementation Pitfalls**: Common mistakes and proper solutions

## Prerequisites

- Python 3.8+
- LangChain and DeepAgents libraries
- OpenAI or Anthropic API key
- Basic understanding of AI agents and tool calling

## Progressive Learning Path

1. **Basic Agent** → Tool calling fundamentals
2. **+Planning** → Proactive task management  
3. **+Filesystem** → Context overflow solutions
4. **+Subagents** → Specialization & isolation
5. **+All Middleware** → Full autonomous workflows
6. **+Caching** → Production optimization
7. **+Proper Implementation** → Avoiding common pitfalls
8. **+Hybrid Filesystem** → Virtual + real persistence

---

**Let's begin our journey!** 🎭

## 1. Setup and Imports

**Goal**: Set up the development environment with all necessary libraries and configurations.

**Why this matters**: Proper setup ensures consistent behavior across all experiments and prevents configuration-related issues.

**Key Components**:
- **LangChain**: Core agent framework
- **DeepAgents**: Advanced agent middleware
- **Model Configuration**: Support for both OpenAI and LiteLLM proxy
- **Custom Tools**: Reusable tools for experiments
- **Debug Utilities**: Enhanced output formatting

In [7]:
%pip install langchain langchain-openai deepagents python-dotenv pydantic

In [9]:
# Install required packages (run this in terminal if needed)

# Core imports
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_core.tools import tool as core_tool

# Deep Agents imports
from deepagents import create_deep_agent, CompiledSubAgent

# Environment and utilities
from dotenv import load_dotenv
import os
import sys
from pydantic import SecretStr
import pprint
import re
import time

# Custom debug formatter for better output
class DebugFormatter:
    def __init__(self, current_experiment=""):
        self.step_counter = 0
        self.original_stdout = sys.stdout
        self.buffer = []
        self.current_experiment = current_experiment  # Accept initial experiment name
        self.execution_timestamps = []
        self.last_execution_time = None
        self.execution_patterns = []

    def set_experiment(self, experiment_name):
        """Set the current experiment name and reset chapter counter"""
        self.current_experiment = experiment_name
        self.step_counter = 0  # Reset chapter counter for new experiment
        self.execution_timestamps = []  # Reset execution tracking
        self.execution_patterns = []  # Reset patterns

    def write(self, text):
        # Capture debug output
        if '[values]' in text or '[updates]' in text:
            self.buffer.append(text)
            # Process complete debug lines
            if text.strip().endswith('}') or len(self.buffer) > 5:
                self.process_debug_output()
                self.buffer = []
        else:
            # Pass through non-debug output
            self.original_stdout.write(text)

    def flush(self):
        self.original_stdout.flush()

    def detect_execution_pattern(self, debug_text):
        """Detect if execution is parallel, concurrent, or sequential"""
        current_time = time.time()

        # Track execution timing
        if self.last_execution_time:
            time_diff = current_time - self.last_execution_time
            self.execution_timestamps.append(time_diff)

        self.last_execution_time = current_time

        # Check for initialization/setup phase
        if 'PatchToolCallsMiddleware' in debug_text:
            self.execution_patterns.append("INITIALIZATION")
            return "🚀 INITIALIZATION", "Setting up agent conversation flow"

        # Check for context processing/thinking phase
        if 'SummarizationMiddleware' in debug_text:
            self.execution_patterns.append("CONTEXT_PROCESSING")
            return "🧠 CONTEXT PROCESSING", "Analyzing and preparing context"

        # Check for tool calls (parallel/sequential execution)
        if 'tool_calls' in debug_text:
            calls_in_step = len(re.findall(r"'name':\s*'task'", debug_text))
            if calls_in_step > 1:
                self.execution_patterns.append("PARALLEL")
                return "🔄 PARALLEL EXECUTION", f"Multiple subagents ({calls_in_step}) working simultaneously"
            elif calls_in_step == 1:
                self.execution_patterns.append("SEQUENTIAL")
                return "➡️ SEQUENTIAL EXECUTION", "Single subagent delegation"

        # Check for subagent completion/results
        if 'ToolMessage' in debug_text and 'tools' in debug_text:
            self.execution_patterns.append("SUBAGENT_WORK")
            return "🔧 SUBAGENT EXECUTION", "Specialized agent processing task"

        # Check for final summarization
        if 'model' in debug_text and 'Here' in debug_text and 'summary' in debug_text.lower():
            self.execution_patterns.append("SYNTHESIS")
            return "✨ SYNTHESIS", "Combining results into final output"

        # Check for concurrent patterns based on timing
        if len(self.execution_timestamps) > 1:
            avg_time_diff = sum(self.execution_timestamps[-3:]) / len(self.execution_timestamps[-3:])
            if avg_time_diff < 0.1:  # Very fast succession
                self.execution_patterns.append("CONCURRENT")
                return "⚡ CONCURRENT EXECUTION", "Rapid sequential operations"
            elif avg_time_diff > 1.0:  # Slower, more deliberate
                self.execution_patterns.append("SEQUENTIAL")
                return "⏳ SEQUENTIAL EXECUTION", "Deliberate step-by-step processing"

        # Default analysis state
        return "� ANALYSIS", "Evaluating task requirements"

    def process_debug_output(self):
        if not self.buffer:
            return

        self.step_counter += 1

        # Detect execution pattern
        full_debug = ''.join(self.buffer)
        pattern_icon, pattern_desc = self.detect_execution_pattern(full_debug)

        # Create a narrative story with experiment name and execution pattern
        experiment_name = self.current_experiment if self.current_experiment else "New Experiment"
        print(f"\n🎭 DEEP AGENT STORY : {experiment_name} - Chapter {self.step_counter}")
        print(f"{pattern_icon} {pattern_desc}")
        print("=" * 50)

        # Parse and create human story
        full_debug = ''.join(self.buffer)

        # Tell the story based on what's happening
        if 'PatchToolCallsMiddleware' in full_debug:
            print("📝 Setting up the conversation...")

        if 'SummarizationMiddleware' in full_debug:
            print("🧠 Preparing context and thinking...")

        if 'tool_calls' in full_debug and 'task' in full_debug:
            calls_count = len(re.findall(r"'name':\s*'task'", full_debug))
            if calls_count > 1:
                print(f"\n🤖 MAIN AGENT: 'This is complex! I'll delegate to {calls_count} specialists simultaneously!'")
                print("   'They can work in parallel to speed things up...'")
            else:
                print("\n🤖 MAIN AGENT: 'This needs expertise. Let me delegate to a specialist...'")

            # Extract task details
            desc_match = re.search(r"'description':\s*'([^']+)'", full_debug)
            if desc_match:
                task_desc = desc_match.group(1)[:120] + "..." if len(desc_match.group(1)) > 120 else desc_match.group(1)
                print(f"   📋 TASK: {task_desc}")

            subagent_match = re.search(r"'subagent_type':\s*'([^']+)'", full_debug)
            if subagent_match:
                subagent_type = subagent_match.group(1)
                print(f"   🎯 CALLING: {subagent_type} subagent")

        if 'tools' in full_debug and 'ToolMessage' in full_debug:
            print("\n🔧 SUBAGENT: 'Working on the research task...'")
            print("   'Let me gather comprehensive information...'")
            print("   'Using web search and analysis tools...'")
            print("   ✅ COMPLETED: Detailed research with milestones table")

        if 'model' in full_debug and 'Here' in full_debug and 'summary' in full_debug.lower():
            print("\n🤖 MAIN AGENT: 'Great work, subagent!'")
            print("   'Now let me create a clean, readable summary for the user...'")
            print("   ✨ FINAL RESULT: Professional summary ready!")

            # Show token usage in a friendly way
            tokens_match = re.search(r"'input_tokens':\s*(\d+).*?'output_tokens':\s*(\d+)", full_debug)
            if tokens_match:
                input_tokens = int(tokens_match.group(1))
                output_tokens = int(tokens_match.group(2))
                print(f"   📊 EFFORT: {input_tokens:,} tokens processed, {output_tokens:,} tokens generated")

        print()  # Add spacing

    def format_values_section(self, values_str):
        print(f"\n📊 CURRENT STATE:")
        try:
            # Simple parsing for messages
            if 'messages' in values_str:
                print(f"   💬 Conversation has {values_str.count('HumanMessage') + values_str.count('AIMessage')} messages")
                if 'HumanMessage' in values_str:
                    print(f"   👤 Latest: User message")
                if 'AIMessage' in values_str:
                    print(f"   🤖 Latest: AI response")
        except:
            print(f"   📄 State data available")

    def format_updates_section(self, updates_str):
        print(f"\n⚡ STATE UPDATES:")

        # Check for different update types
        if 'PatchToolCallsMiddleware' in updates_str:
            print(f"   🔧 MIDDLEWARE: Preparing message flow")

        if 'SummarizationMiddleware' in updates_str:
            print(f"   📝 MIDDLEWARE: Processing context")

        if 'model' in updates_str and 'tool_calls' in updates_str:
            print(f"   🧠 MAIN AGENT DECIDES: Delegating to subagent")
            # Extract task description
            desc_match = re.search(r"'description':\s*'([^']+)'", updates_str)
            if desc_match:
                desc = desc_match.group(1)[:100] + "..." if len(desc_match.group(1)) > 100 else desc_match.group(1)
                print(f"      📋 TASK: {desc}")

            subagent_match = re.search(r"'subagent_type':\s*'([^']+)'", updates_str)
            if subagent_match:
                print(f"      🎯 SUBAGENT: {subagent_match.group(1)}")

        if 'tools' in updates_str and 'task' in updates_str:
            print(f"   🔧 SUBAGENT COMPLETES: Research task finished")
            # Extract content length
            content_match = re.search(r"'content':\s*'([^']{0,200})", updates_str)
            if content_match:
                preview = content_match.group(1).replace('\\n', ' ').strip()
                print(f"      📄 RESULT: {len(updates_str)} chars of detailed research")
                if '|' in preview:  # Looks like a table
                    print(f"      📊 FORMAT: Structured table with milestones")

        if 'model' in updates_str and 'content' in updates_str and 'Here' in updates_str:
            print(f"   ✨ MAIN AGENT SUMMARIZES: Creating final readable response")
            # Extract token usage
            tokens_match = re.search(r"'input_tokens':\s*(\d+).*?'output_tokens':\s*(\d+)", updates_str)
            if tokens_match:
                input_tokens = tokens_match.group(1)
                output_tokens = tokens_match.group(2)
                print(f"      📈 TOKENS: {input_tokens} in → {output_tokens} out")

# Load environment variables
load_dotenv()

print("✅ Environment loaded")
print("🔧 Setting up model configuration...")

# API Key setup (prefer LiteLLM, fallback to OpenAI)
litellm_key = os.getenv("LITELLM_VIRTUAL_KEY")
openai_key = os.getenv("OPENAI_API_KEY")

key = litellm_key or openai_key
if not key:
    raise SystemExit(
        "❌ Missing API key: set LITELLM_VIRTUAL_KEY (for LiteLLM local proxy) or OPENAI_API_KEY in your environment or .env file.\n"
        "Example: export OPENAI_API_KEY=your_key_here"
    )

# Model configuration
base_url = os.getenv("LITELLM_API") if litellm_key else None

try:
    model = ChatOpenAI(
        base_url=base_url,
        api_key=SecretStr(key),
        model=os.getenv("LITELLM_MODEL", "gpt-4o-mini"),
    )
    print("✅ Model initialized successfully")
except Exception as e:
    raise SystemExit(f"❌ Failed to initialize model: {e}")

print("🎯 Setup complete! Ready for experiments.")

In [10]:
# Custom Tools for Experiments

@tool
def search_web(query: str) -> str:
    """Search the web for information - returns mock results for demonstration."""
    print(f"🔍 WEB SEARCH: Querying '{query}'")
    result = f"Mock search results for: {query}\n\n[In a real implementation, this would call a web search API like Tavily or SerpAPI]"
    print(f"🔍 WEB SEARCH: Found results ({len(result)} chars)")
    return result

@tool
def list_directory(path: str = ".") -> str:
    """List files and directories in the specified path."""
    import os
    try:
        items = os.listdir(path)
        result = f"Contents of {path}:\n"
        for item in sorted(items):
            item_path = os.path.join(path, item)
            if os.path.isdir(item_path):
                result += f"📁 {item}/\n"
            else:
                size = os.path.getsize(item_path)
                result += f"📄 {item} ({size} bytes)\n"
        print(f"📂 FILESYSTEM: Listed directory {path}")
        return result
    except Exception as e:
        return f"Error listing directory {path}: {e}"

@tool
def read_file_content(file_path: str) -> str:
    """Read the content of a file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        print(f"📖 FILESYSTEM: Read file {file_path} ({len(content)} chars)")
        return f"Content of {file_path}:\n\n{content}"
    except Exception as e:
        return f"Error reading file {file_path}: {e}"

@tool
def write_file_content(file_path: str, content: str) -> str:
    """Write content to a new file."""
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"✏️ FILESYSTEM: Wrote {len(content)} chars to {file_path}")
        return f"Successfully wrote to {file_path}"
    except Exception as e:
        return f"Error writing to file {file_path}: {e}"

@tool
def edit_file_content(file_path: str, old_string: str, new_string: str) -> str:
    """Edit a file by replacing old_string with new_string."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        if old_string not in content:
            return f"Error: '{old_string}' not found in {file_path}"

        new_content = content.replace(old_string, new_string, 1)

        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(new_content)

        print(f"🔧 FILESYSTEM: Edited {file_path}, replaced '{old_string}' with '{new_string}'")
        return f"Successfully edited {file_path}"
    except Exception as e:
        return f"Error editing file {file_path}: {e}"

@tool
def write_real_file(path: str, content: str) -> str:
    """Write content to a real file on disk (not virtual filesystem)."""
    import os
    try:
        full_path = os.path.abspath(path)
        os.makedirs(os.path.dirname(full_path), exist_ok=True)
        with open(full_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"💾 REAL FILESYSTEM: Wrote {len(content)} chars to {full_path}")
        return f"✅ File written to {full_path}"
    except Exception as e:
        return f"Error writing to real file {path}: {e}"

print("🛠️ Custom tools loaded successfully!")

## 2. Experiment 1: Basic Tool-Calling Agent (Baseline)

**🎯 Goal**: Understand the foundation - simple tool calling without deep agent features.

**❓ Why this matters**: This establishes our baseline understanding of how traditional agents work, highlighting their limitations that deep agents solve.

**🔍 Key Concepts**:
- **Reactive Behavior**: Agent only responds to direct tool calls
- **Single-step Tasks**: Cannot break down complex multi-step problems
- **Context Limitations**: No planning or persistent memory
- **Tool Loop**: Basic request → tool call → response pattern

**❌ Expected Limitations**:
- Cannot handle complex research requiring multiple steps
- No planning or task decomposition
- Context window overflow for large content
- Reactive rather than proactive behavior

**📊 Success Criteria**:
- Agent successfully calls search_web tool
- Returns appropriate mock results
- Demonstrates basic tool-calling capability

In [11]:
print("\n" + "="*80)
print("🚀 BASIC AGENT (create_agent) - Simple Tool Calling")
print("="*80 + "\n")

agent = create_agent(
    model, 
    tools=[search_web]
)

result = agent.invoke(
    {
        "messages": [
            {
                "role": "user", 
                "content": "Research the history of artificial intelligence and summarize the key milestones."
            }
        ]
    }
)

for message in result['messages']:
    if hasattr(message, 'type'):
        msg_type = message.type
    else:
        msg_type = type(message).__name__

    # Human message
    if msg_type == 'human':
        print(f"\n👤 USER: {message.content}")

    # AI message
    elif msg_type == 'ai':
        print(f"\n🤖 AI [{msg_type.upper()}]:")
        if hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"   🛠️  TOOL CALLS:")
            for i, call in enumerate(message.tool_calls, 1):
                tool_name = call['name']
                tool_args = call['args']
                print(f"      {i}. {tool_name.upper()}({tool_args})")
        elif hasattr(message, 'content') and message.content:
            print(f"   💬 {message.content[:200]}{'...' if len(message.content) > 200 else ''}")

    # Tool message
    elif msg_type == 'tool':
        tool_name = getattr(message, 'name', 'unknown_tool')
        print(f"\n🔧 TOOL RESULT [{msg_type.upper()}] - {tool_name}:")
        print(f"   📄 {message.content[:300]}{'...' if len(message.content) > 300 else ''}")

### 📊 Experiment 1 Analysis: Key Insights & Limitations

**✅ What Worked:**
- Agent successfully called the `search_web` tool
- Returned appropriate mock search results
- Demonstrated basic tool-calling capability

**❌ Critical Limitations Identified:**

1. **🔄 Reactive, Not Proactive**
   - Agent only responds to direct requests
   - No initiative to break down complex tasks
   - Single tool call → single response pattern

2. **📝 No Planning Capability**
   - Cannot decompose "research AI history" into manageable steps
   - No todo list or task tracking
   - Treats complex research as a single operation

3. **💭 Context Window Vulnerability**
   - Would overflow with large research content
   - No mechanism to handle long documents
   - Limited to immediate context

4. **🎯 Single-threaded Execution**
   - Cannot manage multi-step workflows
   - No progress tracking or adaptation
   - Fails at complex real-world tasks

**🔑 Key Learning**: Basic agents are reactive tool-callers, not autonomous problem-solvers. They work for simple tasks but fail at complex multi-step problems that require planning, context management, and task decomposition.

**🚀 Next**: Experiment 2 adds planning capability to transform reactive agents into proactive planners.

## 3. Experiment 2: Adding Planning (TodoListMiddleware)

**🎯 Goal**: Transform reactive agents into proactive planners using TodoListMiddleware.

**❓ Why this matters**: Planning is the foundation of autonomous behavior - agents that can break down complex tasks into steps become truly autonomous.

**🔍 Key Concepts**:
- **TodoListMiddleware**: Enables task decomposition and progress tracking
- **Proactive Planning**: Agent creates plans before execution
- **Task Breakdown**: Complex problems split into manageable steps
- **Progress Tracking**: Systematic advancement through planned tasks

**✅ Expected Improvements**:
- Agent creates todo lists for complex tasks
- Breaks down research into specific, actionable steps
- Tracks progress through task completion
- Adapts plans based on new information

**📊 Success Criteria**:
- Agent generates a structured todo list
- Progresses through tasks systematically
- Updates todo status as work completes
- Handles multi-step research effectively

In [5]:
print("\n" + "="*80)
experiment = "(create_deep_agent) - Planning"
print(f"🚀 DEEP AGENT {experiment}")
print("="*80 + "\n")

# Replace stdout with our formatter
debug_formatter = DebugFormatter()
debug_formatter.set_experiment(experiment)
sys.stdout = debug_formatter

planning_agent = create_deep_agent(
    model=model,  # Pass the pre-configured ChatOpenAI model instance
    tools=[search_web],
    system_prompt="""You're a planning agent that breaks down complex tasks into steps.
    For research tasks requiring web search, delegate to your 'researcher' subagent using the task() tool.
    This ensures specialized tools are used effectively.""",

    debug=True  # Enable debug mode to see internal subagent activity
)

result = planning_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Plan my birthday party with a 90s theme. Research popular 90s music, fashion, and food."
        }
    ]
})

# Process final result messages (human-readable summary)
for message in result['messages']:
    if hasattr(message, 'type'):
        msg_type = message.type
    else:
        msg_type = type(message).__name__

    # Human message
    if msg_type == 'human':
        print(f"\n👤 USER: {message.content}")

    # AI message
    elif msg_type == 'ai':
        print(f"\n🤖 AI [{msg_type.upper()}]:")
        if hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"   🛠️  TOOL CALLS:")
            for i, call in enumerate(message.tool_calls, 1):
                tool_name = call['name']
                tool_args = call['args']
                print(f"      {i}. {tool_name.upper()}({tool_args})")
        elif hasattr(message, 'content') and message.content:
            print(f"   💬 {message.content[:200]}{'...' if len(message.content) > 200 else ''}")

    # Tool message
    elif msg_type == 'tool':
        tool_name = getattr(message, 'name', 'unknown_tool')
        print(f"\n🔧 TOOL RESULT [{msg_type.upper()}] - {tool_name}:")
        print(f"   📄 {message.content[:300]}{'...' if len(message.content) > 300 else ''}")


🚀 DEEP AGENT (create_deep_agent) - Planning


🎭 DEEP AGENT STORY : (create_deep_agent) - Planning - Chapter 1
� ANALYSIS Evaluating task requirements



🎭 DEEP AGENT STORY : (create_deep_agent) - Planning - Chapter 2
🚀 INITIALIZATION Setting up agent conversation flow
📝 Setting up the conversation...



🎭 DEEP AGENT STORY : (create_deep_agent) - Planning - Chapter 3
⚡ CONCURRENT EXECUTION Rapid sequential operations



🎭 DEEP AGENT STORY : (create_deep_agent) - Planning - Chapter 4
🧠 CONTEXT PROCESSING Analyzing and preparing context
🧠 Preparing context and thinking...



🎭 DEEP AGENT STORY : (create_deep_agent) - Planning - Chapter 5
➡️ SEQUENTIAL EXECUTION Single subagent delegation

🤖 MAIN AGENT: 'This needs expertise. Let me delegate to a specialist...'
   🎯 CALLING: general-purpose subagent



🎭 DEEP AGENT STORY : (create_deep_agent) - Planning - Chapter 6
➡️ SEQUENTIAL EXECUTION Single subagent delegation

🤖 MAIN AGENT: 'This needs expertise. Let me delegate to a specialist...'


### 📊 Experiment 2 Analysis: The Power of Planning

**✅ Revolutionary Improvements:**

1. **🧠 Proactive Intelligence**
   - Agent now creates plans BEFORE execution
   - Breaks complex tasks into specific, actionable steps
   - Thinks strategically rather than reacting tactically

2. **📋 Systematic Task Management**
   - Generates structured todo lists automatically
   - Progresses through tasks in logical sequence
   - Updates completion status as work advances

3. **🎯 Complex Problem Solving**
   - Handles multi-step research and planning tasks
   - Adapts approach based on intermediate results
   - Maintains coherence across extended workflows

4. **📈 Autonomous Behavior**
   - No longer needs explicit step-by-step instructions
   - Independently manages complex project planning
   - Demonstrates true autonomous agent capabilities

**🔍 Technical Insights:**
- **TodoListMiddleware**: The secret sauce enabling planning
- **Task Decomposition**: Complex problems → manageable steps
- **Progress Tracking**: Systematic advancement through goals
- **Adaptive Planning**: Plans evolve based on new information

**❓ Why This Matters:**
Planning transforms agents from **reactive tool-callers** into **proactive problem-solvers**. This single capability enables:
- Complex project management
- Multi-step research workflows
- Systematic task execution
- Autonomous decision making

**🚀 Next**: Experiment 3 adds filesystem capabilities to solve context overflow problems.

## 4. Experiment 3: Adding Context Management (FilesystemMiddleware)

**🎯 Goal**: Solve context window overflow using FilesystemMiddleware for large content handling.

**❓ Why this matters**: Real-world tasks often involve large datasets and documents that exceed context limits.

**🔍 Key Concepts**:
- **FilesystemMiddleware**: Automatic context offloading
- **Virtual Filesystem**: Persistent storage within agent sessions
- **Large Content Handling**: No more context overflow
- **Intelligent Reading**: Offset/limit for selective access

**✅ Expected Improvements**:
- Handles arbitrarily large content
- Persistent storage across agent reasoning
- Efficient content access patterns
- No context window limitations

In [16]:
print("\n" + "="*80)
experiment = "(CompiledSubAgent) - Subagent"
print(f"🚀 DEEP AGENT {experiment}")
print("="*80 + "\n")

# Replace stdout with our formatter
debug_formatter = DebugFormatter()
debug_formatter.set_experiment(experiment)
sys.stdout = debug_formatter

# Build a dedicated agent graph for the researcher
researcher_graph = create_agent(
    model=model,
    tools=[search_web],
    system_prompt="""You're a research specialist. When given a research task, use your search_web tool to find comprehensive, accurate information.
    Provide detailed findings with sources and key insights."""
)

# Wrap the graph as a compiled subagent
researcher = CompiledSubAgent(
    name="researcher",
    description="Specialized agent for web research with search tools",
    runnable=researcher_graph
)

# Create main agent that can delegate to the researcher subagent
main_agent = create_deep_agent(
    model=model,
    tools=[],  # Main agent delegates via subagents
    subagents=[researcher],
    system_prompt="""You're a project manager coordinating research tasks.
    When you need research done, delegate to your 'researcher' subagent using the task() tool.
    Synthesize findings into actionable insights.""",
    debug=True
)

result = main_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Research the latest developments in quantum computing and summarize the key breakthroughs from 2023-2024."
        }
    ]
})

# Process final result messages (human-readable summary)
for message in result['messages']:
    if hasattr(message, 'type'):
        msg_type = message.type
    else:
        msg_type = type(message).__name__

    # Human message
    if msg_type == 'human':
        print(f"\n👤 USER: {message.content}")

    # AI message
    elif msg_type == 'ai':
        print(f"\n🤖 AI [{msg_type.upper()}]:")
        if hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"   🛠️  TOOL CALLS:")
            for i, call in enumerate(message.tool_calls, 1):
                tool_name = call['name']
                tool_args = call['args']
                print(f"      {i}. {tool_name.upper()}({tool_args})")
        elif hasattr(message, 'content') and message.content:
            print(f"   💬 {message.content[:200]}{'...' if len(message.content) > 200 else ''}")

    # Tool message
    elif msg_type == 'tool':
        tool_name = getattr(message, 'name', 'unknown_tool')
        print(f"\n🔧 TOOL RESULT [{msg_type.upper()}] - {tool_name}:")
        print(f"   📄 {message.content[:300]}{'...' if len(message.content) > 300 else ''}")


### Experiment 3 Analysis: Context Management with FilesystemMiddleware

**What We Learned:**
- **Context Overflow Problem**: Basic agents have limited context windows (4k-8k tokens), making them unsuitable for complex multi-step tasks
- **FilesystemMiddleware Solution**: Provides virtual filesystem for unlimited persistent context storage
- **Built-in Tools**: Automatic access to `ls`, `read_file`, `write_file`, `edit_file`, `write_todos` without custom tool definitions
- **Planning Integration**: Filesystem operations work seamlessly with TodoListMiddleware for organized task execution

**Key Capabilities Demonstrated:**
1. **Persistent Context**: Files can store information beyond conversation limits
2. **Organized Task Management**: Todo lists combined with file operations for complex workflows
3. **Self-Documentation**: Agents can create and maintain their own documentation
4. **Project Exploration**: Systematic analysis of codebases and file structures

**Why This Matters:**
- Enables agents to handle enterprise-scale tasks that require maintaining state across multiple operations
- Solves the fundamental limitation of chat-based interfaces for complex workflows
- Foundation for autonomous software development and maintenance tasks

**Technical Implementation:**
- `create_deep_agent()` with `FilesystemMiddleware` automatically included
- Virtual filesystem persists throughout conversation
- Integration with planning capabilities for structured execution

## 5. Experiment 4: Advanced SubAgent Architecture

### Understanding SubAgents
SubAgents represent a breakthrough in AI agent architecture - they allow a single agent to spawn specialized sub-agents for different aspects of complex tasks. This creates a hierarchical, multi-agent system where:

- **Main Agent**: Acts as project manager, delegating tasks to specialized sub-agents
- **SubAgents**: Focused specialists that handle specific domains (code generation, testing, documentation, etc.)
- **CompiledSubAgent**: Optimized sub-agent implementation for production use

**Why SubAgents Matter:**
- **Specialization**: Each sub-agent can develop expertise in specific domains
- **Parallel Processing**: Multiple sub-agents can work simultaneously on different aspects
- **Scalability**: Complex projects can be broken down into manageable specialized tasks
- **Quality Control**: Domain-specific agents can maintain higher quality standards

**Real-World Applications:**
- Software development teams with specialized roles (architect, developer, tester, documenter)
- Research projects requiring multiple methodologies
- Content creation with different writing styles and expertise areas

In [17]:
print("\n" + "="*80)
experiment = "(FilesystemMiddleware) - Filesystem"
print(f"🚀 DEEP AGENT {experiment}")
print("="*80 + "\n")

# Replace stdout with our formatter
debug_formatter = DebugFormatter()
debug_formatter.set_experiment(experiment)
sys.stdout = debug_formatter

filesystem_agent = create_deep_agent(
    model=model,
    tools=[],  # Built-in filesystem tools automatically available
    system_prompt="""You are a file management planning assistant.
    You have access to built-in filesystem tools for exploring and managing files.

    Your capabilities include:
    - Planning complex file operations and data management tasks
    - Breaking down file-related tasks into organized steps
    - Using filesystem tools to explore project structures
    - Creating summaries and documentation files
    - Organizing and restructuring file hierarchies

    Demonstrate your planning and organizational capabilities for file management tasks.""",
    debug=True
)

result = filesystem_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Explore the current project structure and create a summary file called 'project_summary.txt' with key information about this project."
        }
    ]
})

# Process final result messages (human-readable summary)
for message in result['messages']:
    if hasattr(message, 'type'):
        msg_type = message.type
    else:
        msg_type = type(message).__name__

    # Human message
    if msg_type == 'human':
        print(f"\n👤 USER: {message.content}")

    # AI message
    elif msg_type == 'ai':
        print(f"\n🤖 AI [{msg_type.upper()}]:")
        if hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"   🛠️  TOOL CALLS:")
            for i, call in enumerate(message.tool_calls, 1):
                tool_name = call['name']
                tool_args = call['args']
                print(f"      {i}. {tool_name.upper()}({tool_args})")
        elif hasattr(message, 'content') and message.content:
            print(f"   💬 {message.content[:200]}{'...' if len(message.content) > 200 else ''}")

    # Tool message
    elif msg_type == 'tool':
        tool_name = getattr(message, 'name', 'unknown_tool')
        print(f"\n🔧 TOOL RESULT [{msg_type.upper()}] - {tool_name}:")
        print(f"   📄 {message.content[:300]}{'...' if len(message.content) > 300 else ''}")

# Inspect filesystem data
print(f"\n🔍 FILESYSTEM DATA INSPECTION:")
if 'values' in result and isinstance(result['values'], dict):
    virtual_fs = result['values'].get('filesystem', {})
    if virtual_fs:
        print(f"Found {len(virtual_fs)} virtual files:")
        for path, file_data in virtual_fs.items():
            if isinstance(file_data, dict) and 'content' in file_data:
                print(f"  📄 {path}: {len(file_data['content'])} chars")

### Experiment 4 Analysis: SubAgent Architecture Breakthrough

**What We Learned:**
- **Hierarchical Intelligence**: Main agents can spawn specialized sub-agents for different task domains
- **CompiledSubAgent**: Production-ready sub-agent implementation with optimized performance
- **Parallel Processing**: Multiple sub-agents can work simultaneously on complex projects
- **Specialization Benefits**: Domain-specific agents maintain higher quality and expertise

**Key Capabilities Demonstrated:**
1. **Task Decomposition**: Complex projects broken into specialized components
2. **Quality Assurance**: Each sub-agent focuses on its area of expertise
3. **Scalability**: System can handle arbitrarily complex tasks by adding more specialized agents
4. **Coordination**: Main agent manages workflow and integrates sub-agent outputs

**Why This Matters:**
- **Software Development**: Teams of specialized agents (architect, developer, tester, documenter)
- **Research Projects**: Different methodologies handled by domain experts
- **Content Creation**: Multiple writing styles and expertise areas working together
- **Enterprise Applications**: Complex business processes automated through specialized agents

**Technical Implementation:**
- `CompiledSubAgent` for production use cases
- Automatic tool inheritance and context sharing
- Hierarchical message passing and result aggregation
- Error handling and recovery across agent boundaries

**Real-World Impact:**
This architecture enables AI systems to tackle enterprise-scale problems that were previously impossible, creating virtual teams of AI specialists that can collaborate on complex projects with superhuman coordination and speed.

## 6. Experiment 5: Full Deep Agent Integration

### Combining All Capabilities
This experiment demonstrates the full power of Deep Agents by combining all middleware components:

- **TodoListMiddleware**: For structured task planning and execution
- **FilesystemMiddleware**: For unlimited context and file operations
- **SubAgent Architecture**: For specialized task delegation

**The Complete Agent Stack:**
```
┌─────────────────────────────────────┐
│         MAIN DEEP AGENT             │
│  ┌─────────────────────────────────┐ │
│  │    TodoListMiddleware          │ │
│  │  - Task planning & tracking    │ │
│  └─────────────────────────────────┘ │
│  ┌─────────────────────────────────┐ │
│  │   FilesystemMiddleware         │ │
│  │  - Virtual file persistence    │ │
│  └─────────────────────────────────┘ │
│  ┌─────────────────────────────────┐ │
│  │   SubAgent Architecture        │ │
│  │  - Specialized assistants      │ │
│  └─────────────────────────────────┘ │
└─────────────────────────────────────┘
```

**Real-World Scenario:**
Building a complete software product from concept to deployment, including:
- Requirements analysis and planning
- Architecture design
- Code implementation
- Testing and validation
- Documentation and deployment

In [ ]:
print("\n" + "="*80)
experiment = "(Real Filesystem) - Real Filesystem"
print(f"🚀 DEEP AGENT {experiment}")
print("="*80 + "\n")

# Replace stdout with our formatter
debug_formatter = DebugFormatter()
debug_formatter.set_experiment(experiment)
sys.stdout = debug_formatter

real_filesystem_agent = create_deep_agent(
    model=model,
    tools=[list_directory, read_file_content, write_file_content, edit_file_content, write_real_file],
    system_prompt="""You are a file management expert with access to real filesystem operations.
    You can read, write, and modify actual files on the system.

    Your capabilities include:
    - Reading and analyzing existing files
    - Creating new files with content
    - Editing existing files
    - Managing project structures
    - Writing real files to the filesystem

    Demonstrate your ability to work with real files and create tangible outputs.""",
    debug=True
)

result = real_filesystem_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Read the README.md file in the current project and create a summary file called 'project_analysis.md' with key insights about this project."
        }
    ]
})

# Process final result messages (human-readable summary)
for message in result['messages']:
    if hasattr(message, 'type'):
        msg_type = message.type
    else:
        msg_type = type(message).__name__

    # Human message
    if msg_type == 'human':
        print(f"\n👤 USER: {message.content}")

    # AI message
    elif msg_type == 'ai':
        print(f"\n🤖 AI [{msg_type.upper()}]:")
        if hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"   🛠️  TOOL CALLS:")
            for i, call in enumerate(message.tool_calls, 1):
                tool_name = call['name']
                tool_args = call['args']
                print(f"      {i}. {tool_name.upper()}({tool_args})")
        elif hasattr(message, 'content') and message.content:
            print(f"   💬 {message.content[:200]}{'...' if len(message.content) > 200 else ''}")

    # Tool message
    elif msg_type == 'tool':
        tool_name = getattr(message, 'name', 'unknown_tool')
        print(f"\n🔧 TOOL RESULT [{msg_type.upper()}] - {tool_name}:")
        print(f"   📄 {message.content[:300]}{'...' if len(message.content) > 300 else ''}")

### Experiment 5 Analysis: Full Deep Agent Integration

**What We Learned:**
- **Complete AI Software Company**: Single agent can handle entire development lifecycle
- **Middleware Synergy**: TodoList + Filesystem + SubAgents create superhuman capabilities
- **Enterprise Scale**: Projects that would take development teams can be handled by one agent
- **Quality Assurance**: Built-in testing, documentation, and deployment capabilities

**Key Capabilities Demonstrated:**
1. **End-to-End Development**: From concept to production deployment
2. **Multi-Technology Stack**: Backend, database, containerization, testing, documentation
3. **Quality Control**: Automated testing, validation, and code review processes
4. **Scalability**: Complex projects broken down and executed systematically

**Why This Matters:**
- **Development Acceleration**: What took weeks/months now takes hours
- **Cost Reduction**: Single AI agent replaces entire development teams
- **Consistency**: Uniform code quality and architectural patterns
- **Innovation Speed**: Rapid prototyping and iteration cycles

**Technical Implementation:**
- All middleware components working in harmony
- Custom and built-in tools seamlessly integrated
- SubAgent specialization for different development phases
- Comprehensive project structure generation

**Real-World Impact:**
This represents the future of software development - AI systems that can conceive, design, implement, test, document, and deploy complete software products autonomously. The implications for business, education, and technological progress are profound.

---

## 🎉 Deep Agents Experiment Series Complete!

### Evolution Summary:
1. **Basic Agent**: Simple tool-calling capabilities
2. **Planning Agent**: Structured task execution with TodoListMiddleware
3. **Context Agent**: Unlimited persistence with FilesystemMiddleware
4. **Specialized Agent**: Domain expertise through SubAgent architecture
5. **Complete Agent**: Full-stack development capabilities

### Key Takeaways:
- **Progressive Enhancement**: Each middleware layer adds critical capabilities
- **Composability**: Components work together to create superhuman intelligence
- **Practical Applications**: From simple automation to enterprise software development
- **Future Potential**: Foundation for autonomous AI systems that can solve complex real-world problems

### Next Steps for Exploration:
- Performance optimization and caching strategies
- Multi-agent collaboration patterns
- Integration with external APIs and services
- Real-world deployment and monitoring
- Ethical considerations and safety measures

This notebook demonstrates how Deep Agents evolve from simple assistants to complete AI development systems capable of tackling enterprise-scale challenges.

## 7. Experiment 6: Multiple Parallel Subagents

### Coordinating Team-Based Workflows

This experiment demonstrates the most advanced capability of Deep Agents: **parallel execution of multiple specialized subagents**. Instead of sequential processing, multiple subagents work simultaneously on different aspects of a complex task.

**The Team Approach:**
```
Main Agent (Project Manager)
├── Researcher SubAgent (Parallel)
├── Writer SubAgent (Parallel)
└── Reviewer SubAgent (Parallel)
```

**Key Concepts:**
- **Parallel Execution**: Multiple subagents work simultaneously, not sequentially
- **Specialization**: Each subagent has a specific role and expertise
- **Coordination**: Main agent manages workflow and synthesizes results
- **Efficiency**: Parallel processing dramatically reduces total execution time

**Real-World Applications:**
- Software development teams (architect, developer, tester, documenter)
- Research projects with multiple methodologies
- Content creation with different writing styles and expertise areas
- Complex business processes requiring multiple specialized skills

In [ ]:
print("\n" + "="*80)
experiment = "(Parallel Subagents) - Parallel Subagents"
print(f"🚀 DEEP AGENT {experiment}")
print("="*80 + "\n")

# Replace stdout with our formatter
debug_formatter = DebugFormatter()
debug_formatter.set_experiment(experiment)
sys.stdout = debug_formatter

# Create specialized subagents for different roles
researcher_graph = create_agent(
    model=model,
    tools=[search_web],
    system_prompt="""
You are a research specialist focused on gathering accurate information.
Always use the search_web tool to get current information.
Provide factual, well-researched content only.
"""
)

writer_graph = create_agent(
    model=model,
    tools=[],  # No special tools needed for writing
    system_prompt="""
You are a content writer specializing in creating engaging, well-structured articles.
Focus on clarity, readability, and compelling narratives.
Always structure your writing with clear headings and sections.
"""
)

reviewer_graph = create_agent(
    model=model,
    tools=[],  # No special tools needed for reviewing
    system_prompt="""
You are a content reviewer specializing in quality assurance and improvement.
Check for accuracy, clarity, grammar, and overall quality.
Provide constructive feedback and suggest improvements.
"""
)

# Create CompiledSubAgents for each role
researcher_subagent = CompiledSubAgent(
    name="researcher",
    description="Specialized agent for web research and data gathering",
    runnable=researcher_graph
)

writer_subagent = CompiledSubAgent(
    name="writer",
    description="Specialized agent for content creation and writing",
    runnable=writer_graph
)

reviewer_subagent = CompiledSubAgent(
    name="reviewer",
    description="Specialized agent for content review and quality assurance",
    runnable=reviewer_graph
)

# Create main agent with multiple parallel subagents
parallel_team_agent = create_deep_agent(
    model=model,
    tools=[search_web],  # Main agent can also use search if needed
    subagents=[researcher_subagent, writer_subagent, reviewer_subagent],
    system_prompt="""You are a project manager coordinating a team of specialized AI agents.

You have access to three specialized subagents that can work in parallel:
- 'researcher': Gathers accurate information using web search
- 'writer': Creates engaging, well-structured content
- 'reviewer': Reviews content for quality and provides feedback

For complex tasks, delegate to multiple subagents simultaneously to work in parallel.
Each subagent will provide their specialized contribution, then you synthesize everything into a final result.

IMPORTANT: Use the task() tool to delegate work to subagents. You can call task() multiple times
to create parallel workflows where subagents work simultaneously on different aspects.""",
    debug=True
)

result_parallel = parallel_team_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": """Create a comprehensive guide about 'The Future of Artificial Intelligence in 2025'.

            Delegate to your team of subagents:
            1. Have the researcher gather current trends and predictions about AI in 2025
            2. Have the writer create an engaging article based on the research
            3. Have the reviewer check the article for accuracy and quality

            Coordinate their work to produce a final polished article."""
        }
    ]
})

# Process final result messages (human-readable summary)
for message in result_parallel['messages']:
    if hasattr(message, 'type'):
        msg_type = message.type
    else:
        msg_type = type(message).__name__

    # Human message
    if msg_type == 'human':
        print(f"\n👤 USER: {message.content}")

    # AI message
    elif msg_type == 'ai':
        print(f"\n🤖 AI [{msg_type.upper()}]:")
        if hasattr(message, 'tool_calls') and message.tool_calls:
            print(f"   🛠️  TOOL CALLS:")
            for i, call in enumerate(message.tool_calls, 1):
                tool_name = call['name']
                tool_args = call['args']
                print(f"      {i}. {tool_name.upper()}({tool_args})")
        elif hasattr(message, 'content') and message.content:
            print(f"   💬 {message.content[:200]}{'...' if len(message.content) > 200 else ''}")

    # Tool message
    elif msg_type == 'tool':
        tool_name = getattr(message, 'name', 'unknown_tool')
        print(f"\n🔧 TOOL RESULT [{msg_type.upper()}] - {tool_name}:")
        print(f"   📄 {message.content[:300]}{'...' if len(message.content) > 300 else ''}")

# Analyze parallel execution
subagent_calls = []
for message in result_parallel['messages']:
    if hasattr(message, 'type'):
        msg_type = message.type
    else:
        msg_type = type(message).__name__

    if msg_type == 'ai':
        if hasattr(message, 'tool_calls') and message.tool_calls:
            for call in message.tool_calls:
                if call['name'] == 'task':
                    subagent_calls.append(call)

print(f"\n🔍 PARALLEL EXECUTION ANALYSIS:")
print(f"   📊 Total Subagent Calls: {len(subagent_calls)}")
print(f"   🤖 Subagents Used: researcher, writer, reviewer")

# Analyze timing and coordination
print(f"\n⏱️  EXECUTION PATTERN:")
print(f"   1. Main agent delegates to researcher (gathers data)")
print(f"   2. Main agent delegates to writer (creates content)")
print(f"   3. Main agent delegates to reviewer (quality check)")
print(f"   4. Main agent synthesizes all results")

print(f"\n💡 Key Learning: Multiple subagents can work simultaneously!")
print(f"   - Parallel execution increases efficiency")
print(f"   - Specialization improves quality")
print(f"   - Main agent coordinates the workflow")

### Experiment 6 Analysis: Multiple Parallel Subagents

**What We Learned:**
- **Team Coordination**: Main agent acts as project manager coordinating specialized subagents
- **Parallel Processing**: Multiple subagents work simultaneously on different aspects of complex tasks
- **Specialized Roles**: Each subagent has distinct expertise (research, writing, review)
- **Workflow Orchestration**: Main agent manages delegation, execution, and result synthesis

**Key Capabilities Demonstrated:**
1. **Concurrent Execution**: Multiple subagents running in parallel, not sequentially
2. **Role Specialization**: Each subagent optimized for specific task types
3. **Quality Assurance**: Built-in review process ensures high-quality outputs
4. **Scalability**: Can add more specialized subagents for increasingly complex tasks

**Why This Matters:**
- **Efficiency**: Parallel execution dramatically reduces total processing time
- **Quality**: Specialized agents maintain higher standards in their domains
- **Scalability**: Complex enterprise tasks can be broken into coordinated parallel workflows
- **Human-AI Teams**: Mimics real development teams with specialized roles

**Technical Implementation:**
- Multiple `CompiledSubAgent` instances with different specializations
- Main agent uses `task()` tool to delegate work to subagents
- Parallel execution through concurrent subagent processing
- Result aggregation and synthesis by main agent

**Real-World Impact:**
This architecture enables AI systems to tackle complex, multi-disciplinary projects that would traditionally require entire human teams. The parallel processing capability makes it possible to complete sophisticated tasks in a fraction of the time required by sequential approaches.

---

## 🎉 Complete Deep Agents Experiment Series!

### Evolution Summary:
1. **Basic Agent**: Simple tool-calling capabilities
2. **Planning Agent**: Structured task execution with TodoListMiddleware
3. **Context Agent**: Unlimited persistence with FilesystemMiddleware
4. **Single SubAgent**: Specialized task delegation
5. **Full Integration**: Combined middleware capabilities
6. **Parallel Team**: Multiple subagents working concurrently

### Key Takeaways:
- **Progressive Enhancement**: Each layer adds transformative capabilities
- **Parallel Processing**: Multiple subagents enable team-like coordination
- **Enterprise Scale**: Complex projects become manageable through specialization
- **Future of Work**: AI systems can now coordinate sophisticated multi-agent workflows

### Advanced Applications:
- **Software Development**: Architect, developer, tester, and documenter agents
- **Research Teams**: Multiple methodologies working in parallel
- **Content Production**: Specialized writing, editing, and review agents
- **Business Processes**: End-to-end automation with quality assurance

This comprehensive series demonstrates how Deep Agents evolve from simple assistants into sophisticated AI orchestration systems capable of coordinating complex, multi-disciplinary projects with superhuman efficiency.